In [1]:
import folium
import pandas as pd
import numpy as np
import io
import json
import matplotlib.pyplot as plt
import matplotlib as mpl



In [2]:
base_coordinates = [51.044270, -114.062019]
map = folium.Map(location=base_coordinates)
map

In [3]:
#Here is the series of code requried to print a border polyline

city_boundary = pd.read_csv("City_Boundary_layer.csv")
polygon = city_boundary.iloc[0,1]
polygon = polygon[10:len(polygon)-2]

polygon_list = polygon.split(',')

points = []
for coord in polygon_list:
    longitude ,latitude = coord.strip().split(' ')
    coord_tuple = (float(latitude), float(longitude))
    points.append(coord_tuple)

folium.PolyLine(points, color='blue',weight=2.5, opacity=1).add_to(map)
map

In [ ]:
#TODO: Now we need to find the min and max of the longitude and latidude to define boundaries


In [ ]:
df = pd.DataFrame(points)
min_=df.min(axis=0)
min_

In [ ]:
max_=df.max(axis=0)
max_

In [4]:
def get_geojson_grid(upper_right, lower_left, n=10):
    """Returns a grid of geojson rectangles, and computes the exposure in each section of the grid based on the vessel data.

    Parameters
    ----------
    upper_right: array_like
        The upper right hand corner of "grid of grids" (the default is the upper right hand [lat, lon] of the USA).

    lower_left: array_like
        The lower left hand corner of "grid of grids"  (the default is the lower left hand [lat, lon] of the USA).

    n: integer
        The number of rows/columns in the (n,n) grid.

    Returns
    -------

    list
        List of "geojson style" dictionary objects   
    """

    all_boxes = []

    lat_steps = np.linspace(lower_left[0], upper_right[0], n+1)
    lon_steps = np.linspace(lower_left[1], upper_right[1], n+1)

    lat_stride = lat_steps[1] - lat_steps[0]
    lon_stride = lon_steps[1] - lon_steps[0]

    for lat in lat_steps[:-1]:
        for lon in lon_steps[:-1]:
            # Define dimensions of box in grid
            upper_left = [lon, lat + lat_stride]
            upper_right = [lon + lon_stride, lat + lat_stride]
            lower_right = [lon + lon_stride, lat]
            lower_left = [lon, lat]

            # Define json coordinates for polygon
            coordinates = [
                upper_left,
                upper_right,
                lower_right,
                lower_left,
                upper_left
            ]

            geo_json = {"type": "FeatureCollection",
                        "properties":{
                            "lower_left": lower_left,
                            "upper_right": upper_right
                        },
                        "features":[]}

            grid_feature = {
                "type":"Feature",
                "geometry":{
                    "type":"Polygon",
                    "coordinates": [coordinates],
                }
            }

            geo_json["features"].append(grid_feature)

            all_boxes.append(geo_json)

    return all_boxes

In [5]:
base_coordinates = [51.044270, -114.062019]
map = folium.Map(location=base_coordinates)

lower_left = [50.842822,-114.315796]
upper_right = [51.212425,-113.859905]

grid = get_geojson_grid(lower_left, upper_right, n=10)

for i, geo_json in enumerate(grid):

#     color = plt.cm.Reds(i / len(grid))
#     color = mpl.colors.to_hex(color)
    gj = folium.GeoJson(geo_json)
    popup = folium.Popup("Traffic cameras {}".format(i))
    gj.add_child(popup)

    map.add_child(gj)
map

In [ ]:
traffic_volume = pd.read_csv("Traffic_Volumes_for_2018.csv")
traffic_volume

In [ ]:
traffic_volume=traffic_volume.rename({'multilinestring': 'location'})
traffic_volume.columns=traffic_volume.columns.str.lower()
traffic_volume.head()

In [6]:
lat=[]
long=[]
temp_lat = 51.212425-50.842822
temp_long=-113.859905-(-114.315796)
num=50.842822
num2=-113.859905
for i in range(0,10):
    num+= temp_lat/10
    num2-=temp_long/10
    lat.append(num)
    long.append(num2)
    
long

[-114.2702069,
 -114.2246178,
 -114.1790287,
 -114.1334396,
 -114.0878505,
 -114.0422614,
 -113.9966723,
 -113.9510832,
 -113.9054941]